In [1]:
import math
from typing import Dict, List
import sys
import os

from main import main

/Users/filip/code/cs-294/lab01/main.py:93: SyntaxWarning: invalid escape sequence '\('
  score_message = """


In [2]:
# DO NOT modify this code below.
if __name__ == "__main__":
    assert len(
        sys.argv) > 1, "Please ensure you include a query for some restaurant when executing main."
    # main(sys.argv[1])
    main("What is the overall score for taco bell?") # 

    # "What is the overall score for taco bell?"            3.25    3.05 - 3.45
    # "What is the overall score for In N Out?"             10.000  9.8  - 10.0
    # "How good is the restaurant Chick-fil-A overall?"     10.000  9.8  - 10.0
    # "What is the overall score for Krispy Kreme?"         8.94    8.74 - 9.14
 

[autogen.oai.client: 10-14 16:45:33] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-14 16:45:33] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-14 16:45:33] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-14 16:45:33] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Registering fetch_restaurant_data function
[autogen.oai.client: 10-14 16:45:33] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-14 16:45:33] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Registering calculate_overall_score function
[autogen.oai.clien